In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [2]:
Tx = 862
Ty = int((Tx - 15)/4 + 1)
n_freq = 128

In [3]:
X, Y = np.load('X_train3_mel.npy'), np.load('Y_train3_mel.npy')

In [4]:
"X = " + str(int(X.nbytes / (1024.0 * 1024.0))) + " Mb, " + "Y = " + str(int(Y.nbytes / (1024.0 * 1024.0))) + " Mb"

'X = 2586 Mb, Y = 4 Mb'

In [5]:
# X = X[..., np.newaxis]
X.shape

(3072, 862, 128)

In [6]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(Tx, n_freq)),
        layers.Conv1D(filters=196, kernel_size=15, strides=4),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.Dropout(0.8),
        layers.GRU(units=128, return_sequences = True),
        layers.Dropout(0.8),
        layers.BatchNormalization(),
        layers.GRU(units=128, return_sequences = True),
        layers.Dropout(0.8),
        layers.BatchNormalization(),
        layers.Dropout(0.8),
        layers.TimeDistributed(layers.Dense(1, activation = "sigmoid"))
    ]
)

# model = keras.Sequential(
#     [
#         layers.Conv2D(input_shape=(Tx, n_freq, 1), filters=64, kernel_size=7, strides=1, padding="SAME", activation="relu"),
#         layers.Dropout(0.5),
#         layers.MaxPooling2D(pool_size=3, strides=1, padding="SAME"),
#         layers.BatchNormalization(),
#         layers.Conv2D(filters=64, kernel_size=4, strides=1, padding="SAME", activation="relu"),
#         layers.Dropout(0.5),
#         layers.Flatten(),
#         layers.Dense(32, activation="relu"),
#         layers.Dense(128, activation="relu"),
#         layers.Dense(1, activation="sigmoid")
#     ]
# )

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 212, 196)          376516    
_________________________________________________________________
batch_normalization (BatchNo (None, 212, 196)          784       
_________________________________________________________________
activation (Activation)      (None, 212, 196)          0         
_________________________________________________________________
dropout (Dropout)            (None, 212, 196)          0         
_________________________________________________________________
gru (GRU)                    (None, 212, 128)          125184    
_________________________________________________________________
dropout_1 (Dropout)          (None, 212, 128)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 212, 128)          5

In [11]:
opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [10]:
def load_my_model(condition, model_name):
    if condition:
        print(f"[INFO] You chose to load model = {model_name}...")
        model = keras.models.load_model(model_name)
        print(f"[INFO] Your model was loaded successfully.")
        return model
    else:
        print(f"[INFO] If you want to load the model set condition=True. Model was not loaded.")
        return

In [11]:
model_name = "model-rnn-comrade-44100-mel.v3.h5"
model = load_my_model(True, model_name)
#opt = keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.01)
#model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
model.summary()

[INFO] You chose to load model = model-rnn-comrade-44100-mel.v3.h5...
[INFO] Your model was loaded successfully.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 212, 196)          376516    
_________________________________________________________________
batch_normalization_3 (Batch (None, 212, 196)          784       
_________________________________________________________________
activation_1 (Activation)    (None, 212, 196)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 212, 196)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 212, 128)          125184    
_________________________________________________________________
dropout_5 (Dropout)          (None, 212, 128)          0         
_______

In [9]:
#X_dev, Y_dev = np.load(f'X_dev.npy'), np.load(f'Y_dev.npy')
#X_dev, Y_dev = X_dev[:40], Y_dev[:40]

In [12]:
with tf.device('/gpu:0'):
    model.fit(X, Y, batch_size = 512, epochs=1000)#, validation_data=(X_dev, Y_dev))

Epoch 1/1000
6/6 [==============================] - 2s 257ms/step - loss: 0.4036 - accuracy: 0.9132
Epoch 2/1000
6/6 [==============================] - 2s 253ms/step - loss: 0.4037 - accuracy: 0.9136
Epoch 3/1000
6/6 [==============================] - 2s 253ms/step - loss: 0.4042 - accuracy: 0.9127
Epoch 4/1000
6/6 [==============================] - 2s 253ms/step - loss: 0.4041 - accuracy: 0.9134
Epoch 5/1000
6/6 [==============================] - 2s 253ms/step - loss: 0.4047 - accuracy: 0.9129
Epoch 6/1000
6/6 [==============================] - 2s 252ms/step - loss: 0.4051 - accuracy: 0.9129
Epoch 7/1000
6/6 [==============================] - 2s 254ms/step - loss: 0.4037 - accuracy: 0.9130
Epoch 8/1000
6/6 [==============================] - 2s 253ms/step - loss: 0.4033 - accuracy: 0.9133
Epoch 9/1000
6/6 [==============================] - 2s 255ms/step - loss: 0.4042 - accuracy: 0.9133
Epoch 10/1000
6/6 [==============================] - 2s 254ms/step - loss: 0.4054 - accuracy: 0.9123

In [13]:
!play /home/biowar/handson-ml2/MyFiles/raw_data/COMBINED_ACTIVATIONS/32.wav


/home/biowar/handson-ml2/MyFiles/raw_data/COMBINED_ACTIVATIONS/32.wav:

 File Size: 106k      Bit Rate: 706k
  Encoding: Signed PCM    
  Channels: 1 @ 16-bit   
Samplerate: 44100Hz      
Replaygain: off         
  Duration: 00:00:01.20  

In:100%  00:00:01.20 [00:00:00.00] Out:52.9k [      |      ] Hd:5.8 Clip:0    
Done.


In [14]:
model.save("model-rnn-comrade-44100-mel.v4.h5")
print("[INFO] Model was saved.")

[INFO] Model was saved.
